In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.llms.openai import OpenAI
from langchain_openai import ChatOpenAI

# from  langchain.schema import Document
from llama_index.core.schema import Document
import json
from typing import Iterable

def load_docs_from_jsonl(file_path)->Iterable[Document]:
    array = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            obj = Document(**data, text=data['page_content'])
            array.append(obj)
    return array

docs = load_docs_from_jsonl('./data/E-7 Visa Guide_사증발급관리기관용_한국어.jsonl')

In [9]:
llm = OpenAI(model="gpt-4o", temperature=0)

# Test 1개만
documents = docs[:1]

In [13]:
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core.llama_dataset.generator import DEFAULT_QUESTION_GENERATION_PROMPT


translate_prompt = "\nPlease reply in Korean."
prompt_template_text = DEFAULT_QUESTION_GENERATION_PROMPT
prompt_template_text += translate_prompt

prompt_template = PromptTemplate(prompt_template_text)

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=1,
    text_question_template=prompt_template
)

dataset = dataset_generator.generate_questions_from_nodes()
examples = dataset.examples
for i, example in enumerate(examples):
    contexts = [n[:100] for n in example.reference_contexts]
    print(f"{i + 1}. {example.query}")


1. E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'의 유형에 대해 설명하시오.
2. E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대해 설명하시오.
3. E-7 비자 발급에 있어서의 기본 원칙에 대해 설명하시오.


In [14]:
df = dataset.to_pandas()
df

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'의 유형...,[[**E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상\n직종)...,,None,ai (gpt-4o)
1,E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대해 설명하시오.,[2**](#e-7-사증-소지자들이-취업할-수-있는-분야인-도입직종발급-대상-직종이...,,None,ai (gpt-4o)
2,E-7 비자 발급에 있어서의 기본 원칙에 대해 설명하시오.,[취업 범위)\n19**](#e-7-활동범위근로-범위-취업-범위) \n[**E-7...,,None,ai (gpt-4o)


In [ ]:
dataset.save_json

In [19]:
def save_docs_to_jsonl(array: list, file_path: str) -> None:
    with open(file_path, 'w', encoding='utf-8') as f:
        examples = [el.dict() for el in array]
        data = {
            "examples": examples,
        }
        json.dump(data, f, indent=4, ensure_ascii=False)


dataset_json_file_path = "data/output/test-dataset.json"
save_docs_to_jsonl(examples, dataset_json_file_path)

In [21]:
from llama_index.core.llama_dataset.rag import LabelledRagDataset

dataset = LabelledRagDataset.from_json(dataset_json_file_path)
examples = dataset.examples

examples

[LabelledRagDataExample(query="E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'에 대해 설명하시오.", query_by=CreatedBy(model_name='gpt-4o', type=<CreatedByType.AI: 'ai'>), reference_contexts=["[**E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상\n직종)'이란?\n2**](#e-7-사증-소지자들이-취업할-수-있는-분야인-도입직종발급-대상-직종이란)  \n[**E-7 '도입직종(발급 대상 직종)'의 유형\n3**](#e-7-도입직종발급-대상-직종의-유형)  \n[**E-7 '도입직종(발급 대상 직종)'의 분류,"], reference_answer="E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'에 대해 설명하자면, 이는 E-7 비자를 소지한 외국인들이 한국에서 취업할 수 있는 특정 직종을 의미합니다. '도입직종'은 E-7 비자를 발급받을 수 있는 직종을 지칭하며, 이러한 직종은 한국 정부가 정한 기준에 따라 분류되고 관리됩니다. E-7 비자는 주로 전문직, 기술직, 특정 산업 분야에서 필요한 인력을 대상으로 발급되며, 이를 통해 외국인들이 한국에서 합법적으로 근무할 수 있게 됩니다.", reference_answer_by=CreatedBy(model_name='gpt-4o', type=<CreatedByType.AI: 'ai'>)),
 LabelledRagDataExample(query='E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대해 설명하시오.', query_by=CreatedBy(model_name='gpt-4o', type=<CreatedByType.AI: 'ai'>), reference_contexts=["2**](#e-7-사증-소지자들이-취업할-수-있는-분야인-도입직종발급-대상-직종이란)  \n[**E-7 '도입직종(발급 대상 직종)'의 유형\n3**](#e-7